In [1]:
# Project idea -> add workout counter in look studio google
# Packages required for this project:
import requests
import pandas as pd
import gspread
from df2gspread import df2gspread as d2g
import gspread_dataframe as gd
import pygsheets
from datetime import date
import json

# Package to save datasets to be used in another script for next analysis
from sklearn import datasets

# enableing max columns for dataframes
pd.set_option('display.max_columns', None)

# Importing credentials for Strava's API
from Credentials import StravaCredentials

In [3]:
# Copy and paste this link in your browser and extract code -> #https://www.strava.com/oauth/authorize?client_id=99205&response_type=code&redirect_uri=http://localhost/&approval_prompt=force&scope=profile:read_all,activity:read_all
data = StravaCredentials.data

# Creating date variable
today = date.today().strftime('%B/%m/%Y')

In [4]:
# Getting refresh token
token = requests.post(url= 'https://www.strava.com/api/v3/oauth/token',data=data).json()

# Accessing the token json to get refresh token and access token
refresh_token = token['refresh_token']
access_token = token['access_token']
access_token

'f715442a7a29d8757e1a9ffa36b20dfa6b6c5fc3'

In [5]:
# Getting athlete info
athlete_url = 'https://www.strava.com/api/v3/athlete'
athlete = requests.get(athlete_url+'?access_token='+access_token).json()

# Covnerting json into dataframe and doing some data cleaning
df_athlete = pd.json_normalize(athlete)
df_athlete = df_athlete.T
df_athlete = df_athlete.reset_index()
df_athlete = df_athlete.rename(columns={0:'info'})
df_athlete

,index,info
0,id,58832723
1,username,blindhydra_athlete
2,resource_state,3
3,firstname,Manuel
4,lastname,Elizaldi
5,bio,None
6,city,Austin
7,state,Texas
8,country,United States
9,sex,M


In [6]:
# Setting up url and page
# This API request gives us the list of activities. 
# The table lacks certain details that we will get from another API request
page = 1
url = "https://www.strava.com/api/v3/activities"
access_token = token['access_token']
# Create the dataframe ready for the API call to store your activity data
activities = pd.DataFrame()
while True:
    # get page of activities from Strava
    print('Getting page number:',page)
    r = requests.get(url + '?access_token=' + access_token + '&per_page=200' + '&page=' + str(page))
    r = r.json()
    print(f'Extraction of page {page} Complete')
    # if no results then exit loop
    if (not r):
        print('Extration Done')
        break
    r = pd.json_normalize(r)
    activities = activities.append(r)
    
    page += 1

clean_activities = activities[['id',
'name',
'distance',
'elapsed_time',
'total_elevation_gain',
'sport_type',
'start_date','achievement_count',
'athlete_count',
'start_latlng',
'end_latlng',
'average_speed',
'max_speed',
'average_temp',
'average_heartrate',
'max_heartrate',
'average_cadence',
'elev_high',
'elev_low']]

Getting page number: 1
Extraction of page 1 Complete
Getting page number: 2
Extraction of page 2 Complete
Getting page number: 3
Extraction of page 3 Complete
Getting page number: 4
Extraction of page 4 Complete
Getting page number: 5
Extraction of page 5 Complete
Extration Done


In [7]:
# Checkpoint so I don't have to run API get request
#activities_copy = pd.read_csv(r'C:\Users\Manuel Elizaldi\Desktop\Learning-Testing\StravaAPI\Outputs\clean_activities.csv')

# Creating copy of activitieas dataframe and renaming some columns
general_activities_copy = clean_activities.copy()
general_activities_copy = general_activities_copy.rename(columns={'elapsed_time':'workout_time_sec','sport_type':'workout_type'})
general_activities_copy.head()

,id,name,distance,workout_time_sec,total_elevation_gain,workout_type,start_date,achievement_count,athlete_count,start_latlng,end_latlng,average_speed,max_speed,average_temp,average_heartrate,max_heartrate,average_cadence,elev_high,elev_low
0,8715744482,Afternoon Mountain Bike Ride,19835.5,5718,260.0,MountainBikeRide,2023-03-14T21:55:24Z,22,1,"[30.40146061219275, -97.68447660841048]","[30.40148257277906, -97.68487382680178]",3.962,9.644,22.0,117.6,144.0,NaN,224.0,195.6
1,8710745438,Afternoon Workout,0.0,2790,0.0,Workout,2023-03-13T22:58:33Z,0,1,[],[],0.000,0.000,27.0,127.2,163.0,NaN,NaN,NaN
2,8698197610,Lunch Workout,0.0,1878,0.0,Workout,2023-03-11T18:57:59Z,0,1,[],[],0.000,0.000,29.0,144.2,173.0,NaN,NaN,NaN
3,8693453841,Afternoon Hike,8782.5,11193,452.0,Hike,2023-03-10T21:12:05Z,6,1,"[30.27608041651547, -97.8236420173198]","[30.27573642320931, -97.8233901411295]",1.081,2.262,30.0,113.7,152.0,51.9,275.8,181.6
4,8688907431,Afternoon Mountain Bike Ride,13483.3,4504,160.0,MountainBikeRide,2023-03-09T23:53:44Z,16,1,"[30.401425156742334, -97.68480601720512]","[30.40150453336537, -97.68491330556571]",3.629,7.426,29.0,125.9,156.0,NaN,215.6,188.8


In [8]:
# Cleaning dataframe and converting units 
# Aprox calories burned during workout calculation
# Formual -> CB = T * (0.6309*H + 0.1988*W + 0.2017*A - 55.0969) / 4.184
# CB ->  is the number of calories burned
# T ->  is the duration of exercise in minutes
# H ->  is your average heart rate in beats per minute
# W -> is your weight in kilograms = 80
# A -> is your age in years
general_activities_copy['aprox_calories_burned'] = round((general_activities_copy['workout_time_sec']/60) * ((0.6309*general_activities_copy['average_heartrate']) + (0.1988*80) + (0.2017*26 - 55.0969)) / 4.184,0)

# from meters to kilometers
general_activities_copy[['distance']] = round(general_activities_copy['distance']/1000,2)

# from seconds to minutes
general_activities_copy['workout_time_min'] = round(general_activities_copy['workout_time_sec']/60,2)

# Fix start_date column into the correct format
general_activities_copy[["start_date"]] = pd.to_datetime(general_activities_copy['start_date']).dt.date

# Changing name of workout type => Workout
general_activities_copy['workout_type'] = general_activities_copy['workout_type'].replace({'Workout':'Functional-Cardio Workout'})

# Preview
print('Number of recorded workouts:',len(general_activities_copy))
general_activities_copy.head()

Number of recorded workouts: 646


,id,name,distance,workout_time_sec,total_elevation_gain,workout_type,start_date,achievement_count,athlete_count,start_latlng,end_latlng,average_speed,max_speed,average_temp,average_heartrate,max_heartrate,average_cadence,elev_high,elev_low,aprox_calories_burned,workout_time_min
0,8715744482,Afternoon Mountain Bike Ride,19.84,5718,260.0,MountainBikeRide,2023-03-14,22,1,"[30.40146061219275, -97.68447660841048]","[30.40148257277906, -97.68487382680178]",3.962,9.644,22.0,117.6,144.0,NaN,224.0,195.6,917.0,95.30
1,8710745438,Afternoon Workout,0.00,2790,0.0,Functional-Cardio Workout,2023-03-13,0,1,[],[],0.000,0.000,27.0,127.2,163.0,NaN,NaN,NaN,515.0,46.50
2,8698197610,Lunch Workout,0.00,1878,0.0,Functional-Cardio Workout,2023-03-11,0,1,[],[],0.000,0.000,29.0,144.2,173.0,NaN,NaN,NaN,427.0,31.30
3,8693453841,Afternoon Hike,8.78,11193,452.0,Hike,2023-03-10,6,1,"[30.27608041651547, -97.8236420173198]","[30.27573642320931, -97.8233901411295]",1.081,2.262,30.0,113.7,152.0,51.9,275.8,181.6,1685.0,186.55
4,8688907431,Afternoon Mountain Bike Ride,13.48,4504,160.0,MountainBikeRide,2023-03-09,16,1,"[30.401425156742334, -97.68480601720512]","[30.40150453336537, -97.68491330556571]",3.629,7.426,29.0,125.9,156.0,NaN,215.6,188.8,816.0,75.07


In [9]:
# Creating additional dataframes for specific activities:
# Running type workouts
running_activities = general_activities_copy.loc[general_activities_copy['workout_type'].isin(['Run','TrailRun'])]

# Biking type workouts
biking_activities = general_activities_copy.loc[general_activities_copy['workout_type'].isin(['Ride','MountainBikeRide'])]

# Functional type workouts
functional_activities = general_activities_copy.loc[general_activities_copy['workout_type'].isin(['Functional-Cardio Workout'])]

In [10]:
# Variables for activities breakdown dataframe 
today_msg = f'Total workouts as of {today}'
total_workouts = len(general_activities_copy)

# Breakdown of workout types:
new = [today_msg,total_workouts]
activities_breakdown = general_activities_copy['workout_type'].value_counts().rename_axis('Sport').reset_index(name='Count')
activities_breakdown = activities_breakdown.append(pd.Series(new, index=['Sport','Count']), ignore_index=True)
activities_breakdown

,Sport,Count
0,Functional-Cardio Workout,482
1,Run,60
2,WeightTraining,29
3,Ride,17
4,Swim,16
5,Walk,14
6,MountainBikeRide,10
7,Yoga,5
8,Hike,4
9,TrailRun,3


In [11]:
# General statistics
first_recorded_workout = min(general_activities_copy['start_date'])
most_recent_workout=max(general_activities_copy['start_date'])
average_workout_duration=round(general_activities_copy['workout_time_min'].mean(),2)
average_calories_burned_per_workout=round(general_activities_copy['aprox_calories_burned'].mean(),0)
average_distance_ran=round(running_activities['distance'].mean(),0)
average_biking_distance=round(biking_activities['distance'].mean(),0)

# Storing average calories for later analysis comparing calories from garmin, strava and my calculated value
%store average_calories_burned_per_workout

# Creating dataframe from general statistics variables
# Create the DataFrame
general_stats_df = pd.DataFrame({
    'First Workout':first_recorded_workout,
    'Most Recent Workout': most_recent_workout,
    'Average Workout Duration in Minutes':average_workout_duration,
    'Average Calories Burned Per Workout':average_calories_burned_per_workout,
    'Average Distance Ran in Kilometers':average_distance_ran,
    'Average Biking Distance in Kilometers':average_biking_distance
},index=['Info'])

# Transposing dataframe, setting new index and column
general_stats_df = general_stats_df.T
general_stats_df = general_stats_df.reset_index()
general_stats_df = general_stats_df.rename(columns={'index':'Info','Info':'Data'})
general_stats_df

Stored 'average_calories_burned_per_workout' (float)


,Info,Data
0,First Workout,2020-05-20
1,Most Recent Workout,2023-03-14
2,Average Workout Duration in Minutes,41.34
3,Average Calories Burned Per Workout,511
4,Average Distance Ran in Kilometers,7
5,Average Biking Distance in Kilometers,7


In [12]:
# Creating list of ids of these activities to use in the detailed workout api request 
running_id_list = list(running_activities['id'])
biking_id_list = list(biking_activities['id'])
functional_id_list = list(functional_activities['id'])

In [13]:
# Get detailed view of workouts function:
# This function will get the data for each workout, if it reaches the API request limit it will stop the process
# The API rate limit allows us to do 100 requests for each 15 mintues. To prvent passing this limit we only grab -
# - the most recent 100 workouts from each list.
def GetWorkoutData(workout_list):
    workout_info = []
    workout_num = 1
    if len(workout_list)>100:
        print('This workout list is too large, reducing to the 100 most recent workouts.')
        workout_list = workout_list[:100]
        for i in workout_list:
            print('Extracting workout:', workout_num)
            req = requests.get(url = f'https://www.strava.com/api/v3/activities/{i}?access_token='+access_token)
            if req.status_code == 200:
                req = req.json()
                workout_info.append(req)
                workout_num += 1
            else:
                print('API Rate limit exceeded, stopping extraction')
                break
    else:
        for i in workout_list:
            print('Extracting workout:',workout_num)
            req = requests.get(url = f'https://www.strava.com/api/v3/activities/{i}?access_token='+access_token)
            if req.status_code == 200:
                req = req.json()
                workout_info.append(req)
                workout_num += 1
            else:
                print('API Rate limit exceeded, stopping extraction')
                break


    return workout_info 

In [ ]:
# Test code
#pd.json_normalize(requests.get(url = f'https://www.strava.com/api/v3/activities/8611884034?access_token='+access_token).json())

In [14]:
# Using the function to get a json of the detailed data for functional, running and biking workouts
# Make sure to run one at a time every 15 minutes to not reach API rate limit
#functional_workouts = GetWorkoutData(functional_id_list)
running_workouts = GetWorkoutData(running_id_list)
#biking_workouts = GetWorkoutData(biking_id_list[0:2])

# saving a copy for further analysis
#functional_workouts_json = functional_workouts

Extracting workout: 1
Extracting workout: 2
Extracting workout: 3
Extracting workout: 4
Extracting workout: 5
Extracting workout: 6
Extracting workout: 7
Extracting workout: 8
Extracting workout: 9
Extracting workout: 10
Extracting workout: 11
Extracting workout: 12
Extracting workout: 13
Extracting workout: 14
Extracting workout: 15
Extracting workout: 16
Extracting workout: 17
Extracting workout: 18
Extracting workout: 19
Extracting workout: 20
Extracting workout: 21
Extracting workout: 22
Extracting workout: 23
Extracting workout: 24
Extracting workout: 25
Extracting workout: 26
Extracting workout: 27
Extracting workout: 28
Extracting workout: 29
Extracting workout: 30
Extracting workout: 31
Extracting workout: 32
Extracting workout: 33
Extracting workout: 34
Extracting workout: 35
Extracting workout: 36
Extracting workout: 37
Extracting workout: 38
Extracting workout: 39
Extracting workout: 40
Extracting workout: 41
Extracting workout: 42
Extracting workout: 43
Extracting workout: 

In [15]:
# This function will clean the output from the previous function - GetWorkoutData
def CleanWorkoutDF(workout_json):
    df = pd.json_normalize(workout_json)
    df[['distance']] = round(df['distance']/1000,2)
    df['workout_time_min'] = round(df['elapsed_time']/60,2)
    df[["start_date"]] = pd.to_datetime(df['start_date']).dt.date
    return df[['id',
               'name',
               'start_date',
               'sport_type',
               'distance',
               'workout_time_min',
               'calories',
               'total_elevation_gain',
               'start_latlng',
               'end_latlng',
               'average_speed',
               'max_speed',
               'average_temp',
               'average_heartrate',
               'max_heartrate']]
    

# This function will clean and do some data conversions the json to get the laps from each workout
def GetLapsFromWorkouts(workout_json):
    workout_laps = pd.json_normalize(workout_json,'laps')
    workout_laps = workout_laps[['activity.id','name','elapsed_time','distance','average_heartrate','max_heartrate','average_speed','max_speed']]
    workout_laps = workout_laps.rename(columns={'activity.id':'activity_id',
                                                'name':'lap',
                                                'elapsed_time':'lap_elapsed_time_min',
                                                'distance':'lap_distance',
                                                'average_heartrate':'lap_average_heartrate',
                                                'max_heartrate':'lap_max_heartrate',
                                                'average_speed':'lap_average_speed',
                                                'max_speed':'lap_max_speed'})
    workout_laps['lap_elapsed_time_min'] = round(workout_laps['lap_elapsed_time_min']/60,2)
    workout_laps['lap_distance'] = round(workout_laps['lap_distance']/1000,2)
    return workout_laps

### Create a pivot table from this df 

In [16]:
running_laps = GetLapsFromWorkouts(running_workouts)
running_laps

,activity_id,lap,lap_elapsed_time_min,lap_distance,lap_average_heartrate,lap_max_heartrate,lap_average_speed,lap_max_speed
0,8633145882,Lap 1,6.02,1.00,142.8,163.0,2.77,4.300
1,8633145882,Lap 2,6.35,1.00,159.9,164.0,2.94,4.308
2,8633145882,Lap 3,7.32,1.00,155.3,167.0,2.80,4.417
3,8633145882,Lap 4,6.43,1.00,161.6,169.0,2.76,3.993
4,8633145882,Lap 5,6.97,1.00,155.2,166.0,2.72,3.688
...,...,...,...,...,...,...,...,...
379,3547418838,Lap 1,30.13,3.80,NaN,NaN,2.10,4.500
380,3517497833,Lap 1,28.92,3.57,NaN,NaN,2.06,4.900
381,3509004014,Lap 1,36.27,4.43,NaN,NaN,2.04,4.700
382,3496090180,Lap 1,32.63,3.45,NaN,NaN,1.83,4.800


In [17]:
CleanWorkoutDF(running_workouts)

,id,name,start_date,sport_type,distance,workout_time_min,calories,total_elevation_gain,start_latlng,end_latlng,average_speed,max_speed,average_temp,average_heartrate,max_heartrate
0,8633145882,Afternoon Run,2023-02-27,Run,6.09,39.50,477.0,59.0,"[30.38898054510355, -97.73563295602798]","[30.388930588960648, -97.73604970425367]",2.802,5.188,28.0,156.0,169.0
1,8594917333,Weighted Run,2023-02-20,Run,6.08,43.68,524.0,91.0,"[30.3889175131917, -97.73592129349709]","[30.38914575241506, -97.73582858964801]",2.324,3.717,31.0,158.2,174.0
2,8587667353,Weighted Run,2023-02-19,Run,6.13,41.67,485.0,82.0,"[30.3885461948812, -97.73615305311978]","[30.38905682042241, -97.73590033873916]",2.455,4.272,17.0,145.4,167.0
3,8573739485,Afternoon Run,2023-02-16,Run,5.02,33.63,394.0,53.0,"[30.38702990859747, -97.73675520904362]","[30.388092482462525, -97.73673626594245]",2.835,4.323,16.0,150.9,166.0
4,8538434119,Morning Trail Run,2023-02-10,TrailRun,6.00,88.95,546.0,346.0,"[30.27398812584579, -97.8255036380142]","[30.27507819235325, -97.82482428476214]",1.632,3.908,19.0,137.6,177.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,3547418838,Morning Run,2020-06-01,Run,3.80,30.15,372.9,3.7,"[27.467316, -99.505939]","[27.46671, -99.50669]",2.132,4.500,NaN,NaN,NaN
59,3517497833,Evening Run,2020-05-26,Run,3.57,28.92,352.5,4.2,"[27.46725, -99.505905]","[27.467154, -99.505893]",2.061,4.900,NaN,NaN,NaN
60,3509004014,Morning Run,2020-05-25,Run,4.43,36.25,443.1,15.6,"[27.466956, -99.506607]","[27.467437, -99.506648]",2.065,4.700,NaN,NaN,NaN
61,3496090180,Finished Thesis,2020-05-23,Run,3.45,32.63,340.5,3.9,"[27.466581, -99.505082]","[27.467184, -99.505856]",1.832,4.800,NaN,NaN,NaN


# Laps dataframe ideas:
#### merge the functional workout dataframe to the number of laps per workout. Do this by using a group by (?)
####

In [39]:
# Pending -> add average number of laps
def DescribeWorkoutDF(df):
    average_workout_duration=round(df['workout_time_min'].mean(),2)
    average_calories_burned_per_workout=round(df['calories'].mean(),0)
    average_distance=round(df['distance'].mean(),0)

    # Creating dataframe from general statistics variables
    # Create the DataFrame
    general_stats_df = pd.DataFrame({
        'Average Workout Duration in Minutes':average_workout_duration,
        'Average Calories Burned Per Workout':average_calories_burned_per_workout,
        'Average Distance in Kilometers':average_distance
    },index=['Info'])

    # Transposing dataframe, setting new index and column
    general_stats_df = general_stats_df.T
    general_stats_df = general_stats_df.reset_index()
    general_stats_df = general_stats_df.rename(columns={'index':'Info','Info':'Data'})
    general_stats_df

,id,name,start_date,sport_type,distance,workout_time_min,calories,total_elevation_gain,start_latlng,end_latlng,average_speed,max_speed,average_temp,average_heartrate,max_heartrate
0,8683360176,Afternoon Workout,2023-03-08,Workout,0.0,107.15,448.0,0,[],[],0.0,0.0,29,106.7,158.0
1,8677673116,Afternoon Workout,2023-03-07,Workout,0.0,48.50,435.0,0,[],[],0.0,0.0,31,127.2,158.0
2,8655132144,Afternoon Workout,2023-03-03,Workout,0.0,37.58,400.0,0,[],[],0.0,0.0,28,140.6,173.0
3,8650266138,Afternoon Workout,2023-03-02,Workout,0.0,57.88,561.0,0,[],[],0.0,0.0,29,146.1,181.0
4,8644587607,Afternoon Workout,2023-03-01,Workout,0.0,36.90,367.0,0,[],[],0.0,0.0,30,148.2,177.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,8083176906,Lunch Workout,2022-11-07,Workout,0.0,21.15,255.0,0,[],[],0.0,0.0,31,150.6,174.0
75,8079471189,Lunch Workout,2022-11-06,Workout,0.0,61.37,499.0,0,[],[],0.0,0.0,31,133.2,165.0
76,8068964123,Afternoon Workout,2022-11-04,Workout,0.0,54.23,447.0,0,[],[],0.0,0.0,32,131.6,173.0
77,8068964116,Evening Workout,2022-11-03,Workout,0.0,34.80,370.0,0,[],[],0.0,0.0,29,140.4,172.0


# Checkpoint -> connect to google drive api -> then connect to google looker

#### article in stackoverflow to set up function to upload data to googlesheet:
#### https://stackoverflow.com/questions/62917910/how-can-i-export-pandas-dataframe-to-google-sheets-using-python

In [ ]:
# This function uses gspread and pygsheets modules to upload data to google sheets
def WriteToGsheet(service_file_path, spreadsheet_id, sheet_name, data_df):
    """
    this function takes data_df and writes it under spreadsheet_id
    and sheet_name using your credentials under service_file_path
    """
    gc = pygsheets.authorize(service_file=service_file_path)
    sh = gc.open_by_key(spreadsheet_id)
    try:
        sh.add_worksheet(sheet_name)
    except:
        pass
    wks_write = sh.worksheet_by_title(sheet_name)
    wks_write.clear('A1',None,'*')
    wks_write.set_dataframe(data_df, (1,1), encoding='utf-8', fit=True)
    wks_write.frozen_rows = 1
    
# Setting up parameters for write_to_gsheet function
service_file_path = r'C:\Users\Manuel Elizaldi\Desktop\Learning-Testing\StravaAPI\Credentials\pacific-castle-303123-909a5ddcda92.json'
spreadsheet_id = '1pomkAzlndHBl_czERrwKkoZFUkJRGFjyhRTeoWA6CS4'

In [ ]:
# Uploading functional workouts to google sheets:
functional_workouts_sheet = 'specific_data' 
WriteToGsheet(service_file_path,spreadsheet_id,functional_workouts_sheet,functional_workouts_df)

In [ ]:
# Uploading general stats dataframe to google sheets:
general_stats_sheet = 'General_Statistics' 
WriteToGsheet(service_file_path,spreadsheet_id,general_stats_sheet,general_stats_df)

In [ ]:
# Uploading activities breakdown to google sheets:
activities_breakdown_sheet = 'Activities_Breakdown'
WriteToGsheet(service_file_path,spreadsheet_id,activities_breakdown_sheet,activities_breakdown)

In [ ]:
#creating checkpoint !
# In this example, the my_list variable represents the list that we want to save as a JSON file. We use the open() function to open a new file called "my_list.json" in write mode, and then use the json.dump() method to write the list to the file in JSON format.
with open('functional_workouts_json.json','w') as f:
    json.dump(functional_workouts_json,f)
# The with statement is used here to automatically close the file when we're done writing to it. The "w" parameter in the open() function specifies that we're opening the file in write mode. If the file already exists, its contents will be overwritten. If it doesn't exist, a new file will be created.
# f = open(r'C:\Users\Manuel Elizaldi\Desktop\Learning-Testing\StravaAPI\Outputs\functional_workouts_json.json')
# functional_workouts_json = json.load(f)